In [1]:
VAMPIRE_PATH = '/home/apluska/.vampire/bin/vampire_z3_rel_static_casc2023_6749'
TPTP_PATH = '/home/apluska/TPTP-v8.2.0/'

We select problems which are at most 100_000 bytes in size, have functions with at most arity 8, and at most 16 functions of each arity. 

In [2]:
from foreduce.tptp.parser import read_file
import os
from tqdm.auto import tqdm
from itertools import zip_longest


total, success = 0, 0
num_variables, num_functions = 0, []
for dir, file in (pbar := tqdm([(dir, file) for dir in sorted(os.listdir(TPTP_PATH + 'Problems')) for file in sorted(os.listdir(TPTP_PATH + 'Problems/' + dir))])):
    current = file
    pbar.set_description(f'Selected {success}/{total} Problems, parsing {dir}/{file}')
    if not file.endswith('.p'):
        continue
    try:
        total += 1
        problem = read_file(TPTP_PATH + 'Problems/' + dir + '/' + file, include_path=TPTP_PATH, max_size=100_000)
        _variables = max(len(clause.variables()) for clause in problem.clauses)
        num_variables = max(num_variables, _variables)
        _symbols = []
        for s in problem.function_symbols() | problem.predicate_symbols():
            if s.arity > 8:
                break
            if len(_symbols) <= s.arity:
                _symbols += [0 for _ in range(s.arity + 1 - len(_symbols))]
            _symbols[s.arity] += 1
        else:
            if any(count > 16 for count in _symbols):
                continue
            num_functions = [max(a, b) for a, b in zip_longest(num_functions, _symbols, fillvalue=0)]
            success += 1
            os.makedirs('./problems/' + dir, exist_ok=True)
            with open('./problems/' + dir + '/' + file, 'w') as f:
                f.write(problem.to_tptp())
    except Exception as e:
        continue

print(f'Maximum number of variables: {num_variables}')
print(f'Maximum number of functions of each arity: {num_functions}')

with open('problems/stats.txt', 'w') as f:
    f.write(f'Total problems: {total}\n')
    f.write(f'Successfully parsed problems: {success}\n')
    f.write(f'Maximum number of variables: {num_variables}\n')
    f.write(f'Maximum number of functions of each arity: {num_functions}\n')

/home/apluska/miniconda3/envs/foreduce/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Selected 0/12 Problems, parsing AGT/AGT007+1.p:   0%|          | 12/25963 [00:01<38:06, 11.35it/s]

4906/25963 have been selected. The maximum number of functions of each respective arity is [16, 16, 16, 16, 9, 4, 3, 4, 5].

Next, we generate proofs for these problems using vampire with a timeout of 1. We limit ourselves to proofs which are at most 1_000_000 characters long.

In [ ]:
import subprocess

with open('problems.txt', 'r') as f:
    problems = f.read().split('\n')

total, success = 0, 0
for dir, file in (pbar := tqdm([(dir, file) for dir in sorted(os.listdir('./problems')) for file in sorted(os.listdir('./problems/' + dir))])):
    pbar.set_description(f'Succesfully proved {success}/{total} Problems, proving {dir}/{file}')
    args = [VAMPIRE_PATH, './problems/' + dir + '/' + file,  '--show_new', 'on', '-t', '1', '--avatar', 'off', '--proof', 'off']
    try:
        result = subprocess.run(args, capture_output=True, text=True, timeout=5)
    except subprocess.TimeoutExpired:
        continue
    if result.returncode == 0:
        if 'Refutation found.' in result.stdout:
            success += 1    
            os.makedirs('./proofs/' + dir, exist_ok=True)
            with open('./proofs/' + dir + '/' + file, 'w') as f:
                f.write(result.stdout)
    total += 1

total, success

Succesfully proved 0/0 Problems, proving ALG/ALG002-1.p:   0%|          | 0/4906 [00:00<?, ?it/s]

Succesfully proved 1807/4902 Problems, proving TOP/TOP022+1.p: 100%|██████████| 4906/4906 [53:41<00:00,  1.52it/s]        


(4903, 1807)

We manage to prove about half the selected problems.

Let's bring our proofs into tensor form. We fix a maximum number of 1024 steps per proof and 128 tokens per clause. We will be generating 32 data points per problem, i.e. 4MB of data.

In [ ]:
num_functions = [16, 16, 16, 16, 9, 4, 3, 4, 5]

import os
import torch
from tqdm.auto import tqdm

from foreduce.data.data import VampireProofs
from foreduce.transformer.tokenizer import TokenConfig
from foreduce.vampire.parser import read_file

config = TokenConfig(num_functions=num_functions)
dataset = VampireProofs(config=config, max_steps=1024, max_tokens=128)

datapoints_per_proof = 64

for dir, file in (pbar := tqdm([(dir, file) for dir in sorted(os.listdir('./proofs')) for file in sorted(os.listdir('./proofs/' + dir))])):
    pbar.set_description(f'Parsing proof {dir}/{file}')
    problem, tree = read_file('./proofs/' + dir + '/' + file)
    for i in range(datapoints_per_proof):
        pbar.set_description(f'Converting proof of {dir}/{file} to {i+1}/{datapoints_per_proof} datapoints')
        dataset.add_proof(problem, tree, goal='random')

torch.save(dataset, './proofs.pt')

print("Maximal encountered variables in a clause: {config.num_variables}")

Parsing proof ALG/ALG002-1.p:   0%|          | 0/1807 [00:00<?, ?it/s]

Converting proof of SET/SET578+3.p to 54/64 datapoints:  68%|██████▊   | 1224/1807 [11:27:51<5:27:37, 33.72s/it]    


KeyboardInterrupt: 

In [ ]:
len(dataset)

78389

In [ ]:
import torch
from foreduce.data.data import VampireProofs

dataset = torch.load('./proofs.pt')

In [ ]:
dataset.config

TokenConfig(RESERVED_TOKENS=9, reserved_token_mapping={'<PAD>': 0, '<START>': 1, '<END>': 2, '|': 3, '~': 4, '$true': 5, '$false': 6, 'eq': 7}, num_functions=[16, 16, 16, 16, 9, 4, 3, 4, 5], num_variables=10, embed_dim=128)

In [ ]:
print(f'Size of proofs.pt: {os.path.getsize("./proofs.pt") / 1024**3:.2f} GB')

Size of proofs.pt: 56.97 GB


In [ ]:
embedding(dataset[:10][2].unsqueeze(2))

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [ ]:
import torch.nn as nn

pool = nn.AdaptiveAvgPool2d((1, 128))
pool(embedding(dataset[:10][0])).squeeze(2)

tensor([[[ 3.7955e-02,  1.6495e-02, -4.3709e-02,  ...,  5.4811e-01,
           2.7829e+00,  1.0368e-01],
         [ 2.7564e-02,  2.7573e-02, -4.4436e-02,  ...,  5.6219e-01,
           2.7765e+00,  1.0271e-01],
         [ 2.5892e-02,  7.8649e-02, -1.6624e-02,  ...,  5.1412e-01,
           2.6276e+00,  7.5360e-02],
         ...,
         [ 4.5653e-02,  1.5125e-02, -8.3644e-03,  ...,  6.0137e-01,
           2.6398e+00,  3.7518e-02],
         [ 2.0547e-02,  3.6807e-02, -4.1588e-02,  ...,  5.8795e-01,
           2.4859e+00,  3.0267e-02],
         [ 1.6676e-02,  2.1576e-02,  1.3228e-02,  ...,  5.8944e-01,
           2.5555e+00,  4.3205e-02]],

        [[-1.6349e-02,  1.1977e-02, -3.1936e-02,  ...,  5.4543e-01,
           2.7698e+00,  8.2195e-02],
         [-1.8308e-02, -4.4173e-02,  1.2420e-02,  ...,  5.3383e-01,
           2.7574e+00,  7.8418e-02],
         [-1.8347e-02,  4.8654e-02, -3.6250e-02,  ...,  5.6356e-01,
           2.6051e+00,  6.3861e-02],
         ...,
         [-4.1270e-02, -1

In [ ]:
inverted = {v: k for k, v in mapping.items()}

input = [inverted[i] if i in inverted else f"X{i}" for i in goal.tolist()]
result = ""
for i in range(input):
    result += input[i]
    if input[i+1] not in ["(", ")"]:
        result += " "

'<START>achievable(west(m(s(s(X12())))c(s(s(s(s(X12()))))))boatonwest()east(m(X20())c(X23())))<END>'

Now it's time to extract data from the proofs. Again, we only go for proofs with less than 1_000_000 bytes.

In [ ]:
from foreduce.vampire.parser import parse_string
import os
from tqdm.auto import tqdm

success, total = 0, 0
attempts = []
for dir in (pbar := tqdm(os.listdir('./proofs/'))):
    for file in os.listdir('./proofs/' + dir):
        pbar.set_description(f'Parsed {success}/{total}, curently parsing {dir}/{file}')
        total += 1
        if os.path.getsize('./proofs/' + dir + '/' + file) > 1_000_000:
            continue
        with open('./proofs/' + dir + '/' + file, 'r') as f:
            problem = f.read()
        success += 1
        attempts.append(parse_string(problem))


Parsed 29/43, curently parsing RNG/RNG011-5.p: 100%|██████████| 1/1 [00:21<00:00, 21.48s/it]


In [ ]:
from torchtune.modules import RotaryPositionalEmbeddings
import torch

rotary = RotaryPositionalEmbeddings(4, base=50)
x = torch.ones(2, 2, 4, 4)
rotary(x)


tensor([[[[ 1.0000,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  1.0000]],

         [[-0.3012,  1.3818,  0.8491,  1.1310],
          [-0.3012,  1.3818,  0.8491,  1.1310],
          [-0.3012,  1.3818,  0.8491,  1.1310],
          [-0.3012,  1.3818,  0.8491,  1.1310]]],


        [[[ 1.0000,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  1.0000]],

         [[-0.3012,  1.3818,  0.8491,  1.1310],
          [-0.3012,  1.3818,  0.8491,  1.1310],
          [-0.3012,  1.3818,  0.8491,  1.1310],
          [-0.3012,  1.3818,  0.8491,  1.1310]]]])

In [1]:
from foreduce.transformer.model import Model
from foreduce.data.data import VampireProofs

/home/apluska/miniconda3/envs/foreduce/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

dataset = torch.load('./proofs_test.pt')

In [3]:
model = Model({
        "clause_length": 128,
        "clause_num_heads": 4,
        "clause_embed_layers": 4,
        "clause_embed_dim": 64,
        "problem_length": 1024,
        "problem_num_heads": 8,
        "problem_embed_layers": 8,
        "problem_embed_dim": 512,
    },
    dataset.config
)

In [4]:
model(dataset[:10][0]).shape

torch.Size([10, 1024])

In [5]:
torch.nn.functional.binary_cross_entropy_with_logits(model(dataset[:10][0]), dataset[:10][1].to(torch.float))

tensor(14.5000, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [6]:
dataset[:10][1]

tensor([[1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [1, 0, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0]], dtype=torch.int32)